In [1]:
import pandas as pd
df = pd.read_csv("datasets/cleaned/evaluation_fire_coordinates_date_feat_eng_2.csv", na_values="", 
                                         dtype={"ID_UEV": str}, )
df.dtypes

ID_UEV                             object
MUNICIPALITE                        int64
ETAGE_HORS_SOL                    float64
NOMBRE_LOGEMENT                   float64
CODE_UTILISATION                    int64
LIBELLE_UTILISATION                object
CATEGORIE_UEF                      object
SUPERFICIE_TERRAIN                  int64
SUPERFICIE_BATIMENT               float64
NO_ARROND_ILE_CUM                  object
AGE_BATIMENT                      float64
RATIO_SURFACE                     float64
DENSITE_LOGEMENT                  float64
HAS_MULTIPLE_LOGEMENTS              int64
FIRE_FREQUENCY_ZONE               float64
fire_date                          object
NOMBRE_UNITES                     float64
fire                                 bool
LONGITUDE                         float64
LATITUDE                          float64
fire_month                        float64
fire_year                         float64
fire_season                        object
year_month                        

In [2]:
df.head()

,ID_UEV,MUNICIPALITE,ETAGE_HORS_SOL,NOMBRE_LOGEMENT,CODE_UTILISATION,LIBELLE_UTILISATION,CATEGORIE_UEF,SUPERFICIE_TERRAIN,SUPERFICIE_BATIMENT,NO_ARROND_ILE_CUM,...,fire_year,fire_season,year_month,FIRE_COUNT_LAST_YEAR_ZONE,BUILDING_COUNT,FIRE_RATE_ZONE,FIRE_COUNT_LAST_YEAR_ZONE_NORM,FIRE_RATE_ZONE_NORM,had_fire,missing_coords
0,1038405,50,1.0,1.0,1921,Stationnement intérieur (condo),Condominium,2,16.0,REM19,...,NaN,NaN,NaT,9307,85689,0.108614,1.000000,0.846896,0,False
1,5213144,50,1.0,1.0,1000,Logement,Condominium,69,68.0,REM31,...,NaN,NaN,NaT,701,22003,0.031859,0.057806,0.143310,0,True
2,1036349,50,3.0,1.0,1000,Logement,Condominium,133,127.0,REM19,...,NaN,NaN,NaT,9307,85689,0.108614,1.000000,0.846896,0,True
3,5189527,50,1.0,1.0,1000,Logement,Condominium,16,62.0,REM19,...,NaN,NaN,NaT,9307,85689,0.108614,1.000000,0.846896,0,True
4,1037334,50,1.0,1.0,1000,Logement,Condominium,16,82.0,REM19,...,2023.0,Winter,2023-01,9307,85689,0.108614,1.000000,0.846896,1,False


In [3]:
df["fire_date"] = pd.to_datetime(df["fire_date"], errors="coerce")
df["fire_month"] = df["fire_date"].dt.month.fillna(13).astype(int)
df.head()

,ID_UEV,MUNICIPALITE,ETAGE_HORS_SOL,NOMBRE_LOGEMENT,CODE_UTILISATION,LIBELLE_UTILISATION,CATEGORIE_UEF,SUPERFICIE_TERRAIN,SUPERFICIE_BATIMENT,NO_ARROND_ILE_CUM,...,fire_year,fire_season,year_month,FIRE_COUNT_LAST_YEAR_ZONE,BUILDING_COUNT,FIRE_RATE_ZONE,FIRE_COUNT_LAST_YEAR_ZONE_NORM,FIRE_RATE_ZONE_NORM,had_fire,missing_coords
0,1038405,50,1.0,1.0,1921,Stationnement intérieur (condo),Condominium,2,16.0,REM19,...,NaN,NaN,NaT,9307,85689,0.108614,1.000000,0.846896,0,False
1,5213144,50,1.0,1.0,1000,Logement,Condominium,69,68.0,REM31,...,NaN,NaN,NaT,701,22003,0.031859,0.057806,0.143310,0,True
2,1036349,50,3.0,1.0,1000,Logement,Condominium,133,127.0,REM19,...,NaN,NaN,NaT,9307,85689,0.108614,1.000000,0.846896,0,True
3,5189527,50,1.0,1.0,1000,Logement,Condominium,16,62.0,REM19,...,NaN,NaN,NaT,9307,85689,0.108614,1.000000,0.846896,0,True
4,1037334,50,1.0,1.0,1000,Logement,Condominium,16,82.0,REM19,...,2023.0,Winter,2023-01,9307,85689,0.108614,1.000000,0.846896,1,False


In [4]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from lightgbm import LGBMClassifier

# --- Load dataset ---
df = pd.read_csv("datasets/cleaned/evaluation_fire_coordinates_date_feat_eng_2.csv")

# --- Assign fire_month (13 = no fire) ---
df["fire_date"] = pd.to_datetime(df["fire_date"], errors="coerce")
df["fire_month"] = df["fire_date"].dt.month.fillna(13).astype(int)  # 1–12 = fire month, 13 = no fire

# --- Feature selection ---
features = [
    "ETAGE_HORS_SOL", "NOMBRE_LOGEMENT", "AGE_BATIMENT",
    "SUPERFICIE_TERRAIN", "SUPERFICIE_BATIMENT", "LONGITUDE", "LATITUDE"
]

df["AGE_BATIMENT"] = pd.to_numeric(df["AGE_BATIMENT"], errors="coerce").fillna(0).astype(int)
X = df[features]
y = df["fire_month"]

# --- Split data ---
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

# --- Train model ---
model = LGBMClassifier(objective="multiclass", num_class=13, random_state=42)
model.fit(X_train, y_train)

# --- Evaluate ---
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred, digits=3))


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002330 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1232
[LightGBM] [Info] Number of data points in the train set: 531026, number of used features: 6
[LightGBM] [Info] Start training from score -3.201423
[LightGBM] [Info] Start training from score -3.333060
[LightGBM] [Info] Start training from score -3.197038
[LightGBM] [Info] Start training from score -3.129453
[LightGBM] [Info] Start training from score -3.057336
[LightGBM] [Info] Start training from score -3.274240
[LightGBM] [Info] Start training from score -3.397299
[LightGBM] [Info] Start training from score -3.431007
[LightGBM] [Info] Start training from score -3.488936
[LightGBM] [Info] Start training from score -3.360506
[LightGBM] [Info] Start training from score -3.393592
[LightGBM] [Info] Start training from score -3.394939

In [5]:
print(df["fire_month"].value_counts(normalize=True).sort_index())


fire_month
1     0.040705
2     0.035683
3     0.040884
4     0.043742
5     0.047012
6     0.037845
7     0.033464
8     0.032354
9     0.030534
10    0.034718
11    0.033588
12    0.033543
13    0.555929
Name: proportion, dtype: float64


## Results above are terrible.

## Try dropping non fire rows from the training set

In [6]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from lightgbm import LGBMClassifier

# --- Load dataset ---
df = pd.read_csv("datasets/cleaned/evaluation_fire_coordinates_date_feat_eng_2.csv")

# --- Keep only rows where a fire occurred ---
df["fire_date"] = pd.to_datetime(df["fire_date"], errors="coerce")
df = df[df["fire_date"].notna()].copy()  # keep only fire rows

# --- Extract fire month as target ---
df["fire_month"] = df["fire_date"].dt.month.astype(int)  # 1–12 only

# --- Feature selection ---
features = [
    "ETAGE_HORS_SOL", "NOMBRE_LOGEMENT", "AGE_BATIMENT",
    "SUPERFICIE_TERRAIN", "SUPERFICIE_BATIMENT", "LONGITUDE", "LATITUDE"
]

# --- Ensure clean numeric types ---
df["AGE_BATIMENT"] = pd.to_numeric(df["AGE_BATIMENT"], errors="coerce").fillna(0).astype(int)
X = df[features].fillna(0)
y = df["fire_month"]

# --- Train-test split ---
X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, test_size=0.2, random_state=42
)

# --- Train model ---
model = LGBMClassifier(
    objective="multiclass",
    num_class=12,  # only months 1–12
    random_state=42
)
model.fit(X_train, y_train)

# --- Evaluate ---
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred, digits=3))


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000771 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1248
[LightGBM] [Info] Number of data points in the train set: 235813, number of used features: 6
[LightGBM] [Info] Start training from score -2.389652
[LightGBM] [Info] Start training from score -2.521288
[LightGBM] [Info] Start training from score -2.385266
[LightGBM] [Info] Start training from score -2.317681
[LightGBM] [Info] Start training from score -2.245564
[LightGBM] [Info] Start training from score -2.462469
[LightGBM] [Info] Start training from score -2.585527
[LightGBM] [Info] Start training from score -2.619235
[LightGBM] [Info] Start training from score -2.677164
[LightGBM] [Info] Start training from score -2.548734
[LightGBM] [Info] Start training from score -2.581820
[LightGBM] [Info] Start training from score -2.583167